Libraries to inport

In [6]:
from win32com.client import Dispatch, GetActiveObject
from win32com.shell import shell, shellcon
from pathlib import Path
from datetime import date
from shutil import make_archive
import time
import os
import re
import creds

Access the desktop folder

In [2]:
USSER_DESKTOP_PATH = os.path.join(os.environ['USERPROFILE'], 'Desktop')
CSIDL_DESKTOP = shellcon.CSIDL_DESKTOP
USSER_DESKTOP_CSIDL = shell.SHGetFolderPath(0, CSIDL_DESKTOP, None, 0)

# Emails
USER_EMAIL = creds.USER_EMAIL
WMSERVICE_EMAIL = creds.WMSERVICE_EMAIL
XAASIT_EMAIL = creds.XAASIT_EMAIL
XAASIT_FOLDER = 'xaas-it (PDM)'
# Email sub folders
CHECK_ERROR = '1_Check_Error'
TEST_XAASIT = 'Test Xaas'


In [9]:
def get_running_outlook_inst():
    try:
        return GetActiveObject('Outlook.Application')
    except Exception:
        return False

def email_connection(user_email_address:str, sub_folder:str, main_folder = 'Inbox'):
    # Connect to email
    OUTLOOK = get_running_outlook_inst()
    if not get_running_outlook_inst():
        OUTLOOK = Dispatch('Outlook.Application')
    
    for i in range(10): # Try to connect to OUTLOOK NameSpace. Thie loop is needed because of the: "AttributeError - Outlook.Application.GetNameSpace" thet some times appears
        try:            
            OUTLOOK_NAMESPACE = OUTLOOK.GetNameSpace('MAPI')
        except AttributeError as Outlook_NameSpace_err:
            print(f'### Erron on run: {i} -- {Outlook_NameSpace_err}')
        else:
            break
    # Connect to sub email folders
    FOLDER = OUTLOOK_NAMESPACE.Folders(user_email_address).Folders(main_folder).Folders(sub_folder)    
    # Get the emails
    emails = FOLDER.Items
    # Sort messages by ReceivedTime (descending order for most recent)
    emails.Sort('[ReceivedTime]', 1) # xlAscending = 1 # xlDescending = 2
    
    return OUTLOOK, emails

print()

In [16]:
OUTLOOK = get_running_outlook_inst()
if not get_running_outlook_inst():
    OUTLOOK = Dispatch('Outlook.Application')


for myEmailAddress in OUTLOOK.Session.Accounts:
    print(myEmailAddress)
    print(type(myEmailAddress))

email = OUTLOOK.CreateItem(0)  # 0 = create email item


Andrei.Paveliuc@profidata.com
<class 'win32com.client.CDispatch'>
pdmoperations@profidata.com
<class 'win32com.client.CDispatch'>


In [10]:
def email_connection(user_email_address:str, sub_folder:str, main_folder = 'Inbox'):
    # Connect to email
    OUTLOOK = Dispatch('Outlook.Application')
    for i in range(10): # Try to connect to Outlook. Thie loop is needed because of the: "AttributeError - Outlook.Application.GetNameSpace" thet some times appears
        try:
            OUTLOOK_NameSpace = OUTLOOK.GetNameSpace('MAPI')
        except Exception as outlook_NameSpace_err:
            print(f'### Erron on run: {i} -- {outlook_NameSpace_err}')
            time.sleep(1)
        else:
            break

    if user_email_address in [str(folder) for folder in OUTLOOK_NameSpace.Folders]:
        print(f'Yes: {user_email_address}')
    else:
        print(f'No: {user_email_address}')

    for folder in OUTLOOK_NameSpace.Folders:
        print(folder)

    print('---------------------------------')
    
    FOLDER = OUTLOOK_NameSpace.Folders(user_email_address).Folders(main_folder).Folders(sub_folder)
    
    # Get the emails
    emails = FOLDER.Items
    # Sort messages by ReceivedTime (descending order for most recent)
    emails.Sort('[ReceivedTime]', 1) # xlAscending = 1 # xlDescending = 2

    return OUTLOOK, emails

# OUTLOOK, Test_XaasIT_emails = email_connection(USER_EMAIL, TEST_XAASIT)
email_connection(XAASIT_FOLDER, CHECK_ERROR)





Yes: xaas-it (PDM)
Iftaas (PDM)
WMService
xaas-it (PDM)
xaas (PDM)
Andrei.Paveliuc@profidata.com
Public Folders - Andrei.Paveliuc@profidata.com
---------------------------------


(<COMObject Outlook.Application>, <COMObject <unknown>>)

Check the tonnection to eachi folder and sub folder

In [13]:
def email_connection(user_email_address:str, sub_folder:str, main_folder = 'Inbox'):
    # Connect to email
    OUTLOOK = Dispatch('Outlook.Application')
    for i in range(10): # Try to connect to OUTLOOK NameSpace. Thie loop is needed because of the: "AttributeError - Outlook.Application.GetNameSpace" thet some times appears
        try:            
            OUTLOOK_NameSpace = OUTLOOK.GetNameSpace('MAPI')
        except AttributeError as outlook_NameSpace_err:
            print(f'### Erron on run: {i} -- {outlook_NameSpace_err}')
        else:
            break
        print(i)
    # Check if the folder/email is present in Outlook
    if user_email_address in [str(folder) for folder in OUTLOOK_NameSpace.Folders]:
        print(f'Yes: {user_email_address}')
    else:
        print(f'No: {user_email_address}')

    # Check if the inbox folder is pressent 
    if main_folder in [str(folder) for folder in OUTLOOK_NameSpace.Folders(user_email_address).Folders]:
        print(f'Yes: {main_folder}')
    else:
        print(f'No: {main_folder}')

     # Check if the Check_Error folder is pressent 
    if sub_folder in [str(folder) for folder in OUTLOOK_NameSpace.Folders(user_email_address).Folders(main_folder).Folders]:
        print(f'Yes: {sub_folder}')
    else:
        print(f'No: {sub_folder}')
    
    print('---------------------------------')
    print(OUTLOOK_NameSpace.Folders(user_email_address).Folders(main_folder).Folders)
    print('---------------------------------')
    xaassit_subfolders = [str(folder) for folder in OUTLOOK_NameSpace.Folders(user_email_address).Folders(main_folder).Folders]
    print(f'xaassit_subfolders: {xaassit_subfolders}')

    # Connect to sub email folder
    FOLDER = OUTLOOK_NameSpace.Folders(user_email_address).Folders(main_folder).Folders(sub_folder)
    # Get the emails
    emails = FOLDER.Items
    # Sort messages by ReceivedTime (descending order for most recent)
    emails.Sort('[ReceivedTime]', 1) # xlAscending = 1 # xlDescending = 2

    return OUTLOOK, emails

email_connection(XAASIT_FOLDER, CHECK_ERROR)
# email_connection(USER_EMAIL, TEST_XAASIT)




Yes: xaas-it (PDM)
Yes: Inbox
Yes: 1_Check_Error
---------------------------------
<COMObject <unknown>>
---------------------------------
xaassit_subfolders: ['Riskline', 'XDO_Done', 'WM-Service', 'WM_EDDy_Done', 'Wildcard_SSL_Done', 'VisualCron_Done', 'UI_Done', 'SULZER_Done', 'Splunk_Done', 'SLF_Done', 'SKB_Done', 'SKAG_Done', 'RedHat_Done', 'PVB_Done', 'PMG_Done', 'PAX_Done', 'OKAG_Done', 'OeKB_Done', 'NOL_Done', 'MFI_Done', 'LUKB_Done', 'LLB_Done', 'KSKK_Done', 'IFTaas_Done', 'IFTAAS', 'HASPA_Done', 'Everyware_Done', 'EB_Done', 'EAMIS_Done', 'Daily_Copy', 'CSLP_Done', 'CS_Done', 'BNY_Done', 'BLKB_Done', 'BFR_Done', 'Axians_Done', 'AIB_Done', '8_LDAP', '7_Firewall_Alert', '6_Security_Incidents', '5_Check_Client', '4_Helix', '3_Done_general', '2_Check_OK', '1_Check_Error', '0_Prio_0_Alerts']


(<COMObject Outlook.Application>, <COMObject <unknown>>)

In [ ]:
print(OUTLOOK_NameSpace.Accounts[1])
print(OUTLOOK_NameSpace.Folders[3])

    for folder in OUTLOOK_NameSpace.Folders:
        print(folder)
    print('------------------------')
    print(OUTLOOK_NameSpace.Folders[3])
    # print(dir(OUTLOOK_NameSpace.Accounts))
    addrBook = OUTLOOK_NameSpace.Folders.GetPrevious()
    FOLDER = OUTLOOK_NameSpace.Folders(user_email_address).Folders(main_folder).Folders(sub_folder)
    print(f'FOLDER: {OUTLOOK_NameSpace.Folders('Public Folders - xaasit@profidata.com')}')
    print(f'FOLDER: {OUTLOOK_NameSpace.Folders('xaasit@profidata.com').Count}')

In [ ]:
def email_connection(user_email_address:str, sub_folder:str, main_folder = 'Inbox'):
    # Connect to email
    for i in range(10): # Try to connect to Outlook. Thie loop is needed because of the: "AttributeError - Outlook.Application.GetNameSpace" thet some times appears
        try:
            OUTLOOK = Dispatch('Outlook.Application')
            OUTLOOK_NameSpace = OUTLOOK.GetNameSpace('MAPI')
        except AttributeError as outlook_NameSpace_err:
            print(f'### Erron on run: {i} -- {outlook_NameSpace_err}')
        print(i)
    # Connect to sub email folder
    FOLDER = OUTLOOK_NameSpace.Folders(user_email_address).Folders(main_folder).Folders(sub_folder)

    # Get the emails
    emails = FOLDER.Items
    # Sort messages by ReceivedTime (descending order for most recent)
    emails.Sort('[ReceivedTime]', 1) # xlAscending = 1 # xlDescending = 2

    return OUTLOOK, emails


OUTLOOK, Test_XaasIT_emails = email_connection(USER_EMAIL, TEST_XAASIT)
# OUTLOOK, XaasIT_emails = email_connection(XAASIT_EMAIL, CHECK_ERROR)

In [ ]:
def email_connection(user_email_address:str, sub_folder:str, main_folder = 'Inbox'):
    # Connect to email
    for i in range(10): # Try to connect to Outlook. Thie loop is needed because of the: "AttributeError - Outlook.Application.GetNameSpace" thet some times appears
        try:
            OUTLOOK = Dispatch('Outlook.Application')
            OUTLOOK_NameSpace = OUTLOOK.GetNameSpace('MAPI')
        except AttributeError as outlook_NameSpace_err:
            print(f'### Erron on run: {i} -- {outlook_NameSpace_err}')
        except Exception:
            if not is_outlook_running():
                open_outlook()
            OUTLOOK = Dispatch('Outlook.Application')
            OUTLOOK_NameSpace = OUTLOOK.GetNameSpace('MAPI')
        print(i)
    # Connect to sub email folder
    FOLDER = OUTLOOK_NameSpace.Folders(user_email_address).Folders(main_folder).Folders(sub_folder)

    # Get the emails
    emails = FOLDER.Items
    # Sort messages by ReceivedTime (descending order for most recent)
    emails.Sort('[ReceivedTime]', 1) # xlAscending = 1 # xlDescending = 2

    return OUTLOOK, emails

OUTLOOK, Test_XaasIT_emails = email_connection(USER_EMAIL, TEST_XAASIT)
# OUTLOOK, XaasIT_emails = email_connection(XAASIT_EMAIL, CHECK_ERROR)

In [ ]:
    print(OUTLOOK_NameSpace.Folders.Count)
    addrBook = OUTLOOK_NameSpace.Folders(user_email_address).Folders.GetFirst()
    print(addrBook.Name)
    FOLDER = OUTLOOK_NameSpace.Folders(user_email_address).Folders(main_folder).Folders(sub_folder)

In [ ]:
    print(OUTLOOK_NameSpace.Folders.Count)
    addrBook = OUTLOOK_NameSpace.Folders.GetFirst()
    print(addrBook.Name)
    FOLDER = OUTLOOK_NameSpace.Folders(user_email_address).Folders(main_folder).Folders(sub_folder)

In [ ]:
# 1 - C# tot python
# 2 - Ai folosit paranteze pentru proptirtatii
# 3 - Diferenta dintre _Folders si Folders
# 4 - Git
# 5 - 

In [73]:
AI = {'client':'AI',
      'SN_mask':'CHECK_ERROR AI Xentis SN',
      'eMail_To':creds.AI_eMail_To,
      'BBC':'',
      'eMail_subject':'Unprocessed deliveries'}

In [ ]:
from box import Box

AI = Box({
    'client': 'AI',
    'SN_mask': 'CHECK_ERROR AI Xentis SN',
    'eMail_To': '',
    'BBC': '',
    'eMail_subject': 'Unprocessed deliveries'
})

AI.client

In [5]:
from datetime import datetime

DATE = datetime.now()
DATE = DATE.strftime('%Y-%m-%d_%H-%M-%S')

DATE

'2024-12-13_19-11-23'